# Integer to Roman Numerals

Now let's try the following [leetcode problem]([Leetcode](https://leetcode.com/problems/integer-to-roman/)).

Roman numerals are represented by seven different symbols: I, V, X, L, C, D and M.

```
Symbol       Value

I             1
V             5
X             10
L             50
C             100
D             500
M             1000
```

Roman numerals are usually written largest to smallest from left to right. However, the numeral for four is not IIII. Instead, the number four is written as IV. Because the one is before the five we subtract it making four. The same principle applies to the number nine, which is written as IX. There are six instances where subtraction is used:

* I can be placed before V (5) and X (10) to make 4 and 9. 
* X can be placed before L (50) and C (100) to make 40 and 90. 
* C can be placed before D (500) and M (1000) to make 400 and 900.

Given an integer, convert it to a roman numeral.

In [1]:
coins = [1,2, 5]
amount = 11
# output 3,  (5, 5, 1)

### Examples

Ok, so our chart above of roman numeral to number looks like a dictionary.  Let's convert it to one.

* Eg 1

> Small change problem 

In [7]:
numbers = [1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1] # [1, 5, 10, 50, 100, 500, 1000]
numerals = ["M", "CM", "D","CD", "C", "XC", "L","LX", "X", "IX", "V", "IV", "I"]

In [14]:
def num_to_numerals(current_num):
    numerals = []
    while current_num > 0:
        numeral, current_num = subtract_from(current_num)
        numerals.append(numeral)
    return ''.join(numerals)
    
def subtract_from(current_num):
    for idx, number in enumerate(numbers):
            if number <= current_num:
                current_num = current_num - number
                return numerals[idx], current_num

In [16]:
current_num = 1994
num_to_numerals(current_num)

'MCMXCIV'

Nice.

### The long winding road

> This is going to hurt.

Ok, my second thought is that maybe there is a pattern/general rule.  For example, the special circumstances seem to apply in a sort of pattern.  

In [23]:
numbers = [1000,  500,  100,  50,  10, 5, 1] # [1, 5, 10, 50, 100, 500, 1000]
numerals = ["M", "D", "C",  "L", "X",  "V",  "I"]

In [36]:
def subtract_from(current_num):
    int_to_rom = {1: 'I', 10: 'X', 100: 'C'}
    for idx, number in enumerate(numbers):
            diff = current_num - number # -10
            if diff in [-1, -10, -100]:
                return int_to_rom[-diff], current_num + (-diff)
            if number <= current_num:
                current_num = current_num - number
                return numerals[idx], current_num

def num_to_numerals(current_num):
    rom_nums = []
    while current_num > 0:
        rom_num, current_num = subtract_from(current_num)
        rom_nums.append(rom_num)
    return ''.join(rom_nums)

In [32]:
num_to_numerals(90)

'XC'

In [33]:


num_to_numerals(94)

'LXXXXIV'

In [34]:
# 'XCIV'

In [109]:
num = 94 
# desired_output = XCIV

The problem is that we want to break our number of 94 into something like the following.

In [162]:
[90, 4]

[90, 4]

Where the first digit represents the 10s, and the second element equals the number of 1s.  And then we can apply our logic to each digit.

It's not a bad approach, but it seems pretty complicated.

> For example, here's how we could do it.

In [146]:
num = 94
nums = list(str(num))
# evolving our code to get to the right format...
[((len(nums) - idx - 1), int(num)) for idx, num in enumerate(nums)] # [(1, 9), (0, 4)]

[(10**(len(nums) - idx - 1), int(num)) for idx, num in enumerate(nums)] # [(10, 9), (1, 4)]

nums_by_ten = [(10**(len(nums) - idx - 1)*int(num)) for idx, num in enumerate(nums)]
nums_by_ten

[90, 4]

And let's see if our `subtract_from` function can handle these one at a time.

In [147]:
subtract_from(90)

('X', 100)

Ok, so basically we'll have to loop through our list of numbers, but when we use the "subtraction" method, we'll need to prepend to the `nums_by_ten` list.

In [186]:
def subtract_from(current_num, by_ten_idx, nums_by_ten):
    num_to_numeral = {1: 'I', 10: 'X', 100: 'C'}
    for idx, number in enumerate(numbers):
            diff = current_num - number
            if diff in [-1, -10, -100]:
                abs_diff = abs(diff)
                new_num = current_num + abs_diff
                print(new_num)
                nums_by_ten[by_ten_idx] = new_num
                return num_to_numeral[abs_diff]
            if number <= current_num:
                current_num = current_num - number
                nums_by_ten.pop(0)
                return numerals[idx]

In [189]:
numbers = [1000, 500, 100, 50, 10, 5, 1]
numerals = ["M", "D", "C", "L", "X", "V", "I"]

translated_numerals = []
nums_by_ten = [90, 4]
while nums_by_ten:
    print(nums_by_ten)
    for by_ten_idx, num_by_ten in enumerate(nums_by_ten):
        translated_numeral = subtract_from(num_by_ten, by_ten_idx, nums_by_ten)
        translated_numerals.append(translated_numeral)
        print(translated_numerals)
        break
''.join(translated_numerals)

[90, 4]
100
['X']
[100, 4]
['X', 'C']
[4]
5
['X', 'C', 'I']
[5]
['X', 'C', 'I', 'V']


'XCIV'

Ok, looks pretty good.  Let's turn wrap this in functions and try with the example given in the problem: 1994.

In [7]:
numbers = [1000, 500, 100, 50, 10, 5, 1]
numerals = ["M", "D", "C", "L", "X", "V", "I"]

def numbers_by_ten(number):
    nums = list(str(number))
    return [(10**(len(nums) - idx - 1)*int(num)) for idx, num in enumerate(nums)]

# numbers_by_ten(1994) # [1000, 900, 90, 4]

In [8]:
def subtract_from(current_num, by_ten_idx, nums_by_ten):
    num_to_numeral = {1: 'I', 10: 'X', 100: 'C'}
    for idx, number in enumerate(numbers):
            diff = current_num - number
            if diff in [-1, -10, -100]:
                abs_diff = abs(diff)
                new_num = current_num + abs_diff
                nums_by_ten[by_ten_idx] = new_num
                return num_to_numeral[abs_diff]
            if number <= current_num:
                current_num = current_num - number
                if current_num > 0:
                  nums_by_ten[by_ten_idx] = current_num
                  return numerals[idx]
                else:
                  nums_by_ten.pop(0)
                  return numerals[idx]

In [9]:
def translate_to_numerals(number):
    translated_numerals = []
    nums_by_ten = numbers_by_ten(number)
    while nums_by_ten:
        for by_ten_idx, num_by_ten in enumerate(nums_by_ten):
            translated_numeral = subtract_from(num_by_ten, by_ten_idx, nums_by_ten)
            translated_numerals.append(translated_numeral)
            break
    return ''.join(translated_numerals)

In [11]:
translate_to_numerals(1994)

'MCMXCIV'

In [12]:
translate_to_numerals(1984)

'MCMLXXXIV'

Wow, crazy.

### Resources

[Leetcode](https://leetcode.com/problems/integer-to-roman/)